In [2]:
import re

In [46]:
def clean_data(data):
    data = data.replace('\\n', ' ')
    data = " ".join(data.split())
    return data

def check_pdf_pattern(data):
    if re.search('Relatora(.+?)Designada', data) or re.search('Relator(.+?)Designado', data):
        pattern = 'NOT_FOUND'
    else:
        if re.search('Órgão(.+?)Processo N.', data):
            pattern = 'NEW'
        elif re.search('Órgão(.+?)Classe', data):
            pattern = 'OLD'
        else:
            pattern = 'NOT_FOUND'
    return pattern

def extract_orgao(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            orgao = re.search('Órgão(.+?)Processo N.', data).group(1)
        elif pattern == 'OLD':
            orgao = re.search('Órgão(.+?)Classe', data).group(1).replace(':', '')
    except Exception as e:
        pass
    return clean_data(orgao)

def extract_process_number(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
                if re.search('AGRAVANTE(.+?)AGRAVADO', data):
                    process_number = re.search('Processo N.(.+?)AGRAVANTE', data).group(1)
                elif re.search('APELANTE(.+?)APELADO', data):
                    process_number = re.search('Processo N.(.+?)APELANTE', data).group(1)
        elif pattern == 'OLD':
            classe = re.search('Classe(.+?)N. Processo', data).group(1).replace(':', '')
            if re.search('Apelante(.+?)Apelado', data):
                process_number = classe + ' ' +  re.search('N. Processo(.+?)Apelante', data).group(1).replace(':', '') 
            elif re.search('Agravante(.+?)Agravado', data):
                process_number = classe + ' ' +  re.search('N. Processo(.+?)Agravante', data).group(1).replace(':', '') 
    except Exception as e:
        pass
    return clean_data(process_number)

def extract_apelante(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
                if re.search('AGRAVANTE(.+?)AGRAVADO', data):
                    apelante = re.search('AGRAVANTE(.+?)AGRAVADO', data).group(1).replace('(S)', '')
                elif re.search('APELANTE(.+?)APELADO', data):
                    apelante = re.search('APELANTE(.+?)APELADO', data).group(1).replace('(S)', '')
        elif pattern == 'OLD':
            if re.search('Apelante(.+?)Apelado', data):
                apelante = re.search('Apelante(.+?)Apelado', data).group(1).replace('(s)', '').replace(':', '')
            elif re.search('Agravante(.+?)Agravado', data):
                apelante = re.search('Agravante(.+?)Agravado', data).group(1).replace('(s)', '')
    except Exception as e:
        pass
    return clean_data(apelante)

def extract_apelado(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            if re.search('AGRAVADO(.+?)Relator', data):
                apelado = re.search('AGRAVADO(.+?)Relator', data).group(1).replace('(S)', '')
            else:
                apelado = re.search('APELADO(.+?)Relator', data).group(1).replace('(S)', '')
        elif pattern == 'OLD':
            if re.search('AGRAVANTE(.+?)Relator', data):
                apelado = re.search('AGRAVADO(.+?)Relator', data).group(1).replace('(s)', '').replace(':', '')
            else:
                apelado = re.search('Apelado(.+?)Relator', data).group(1).replace('(s)', '').replace(':', '')
    except Exception as e:
        print(e)
        pass
    return clean_data(apelado)

def extract_relator(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            search = "Relatora"
            find_one = re.findall(r"\b" + search + r"\b", data)
            if find_one:
                relator = re.search('Relatora(.+?)Acórdão Nº', data).group(1).replace('Desembargadora ', '')
            else:
                relator = re.search('Relator(.+?)Acórdão Nº', data).group(1).replace('Desembargador ', '')
        elif pattern == 'OLD':
            search = "Relatora"
            find_one = re.findall(r"\b" + search + r"\b", data)
            if find_one:
                regex = r"(?s)(?<=Relatora)(.*?)(?=\\n)"
                matches = re.finditer(regex, data, re.MULTILINE)
                matches_lst = [i.group(0) for i in matches]
                relator = matches_lst[0].replace('Desembargadora ', '').replace(':', '')
            else:
                regex = r"(?s)(?<=Relator)(.*?)(?=\\n)"
                matches = re.finditer(regex, data, re.MULTILINE)
                matches_lst = [i.group(0) for i in matches]
                relator = matches_lst[0].replace('Desembargador ', '').replace(':', '')
    except Exception as e:
        pass
    return clean_data(relator)

def extract_acordao_number(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            acordao_number = re.search('Acórdão Nº(.+?)EMENTA', data).group(1)
        elif pattern == 'OLD':
            regex = r"(?s)(?<=Acórdão N.)(.*?)(?=\\n)"
            matches = re.finditer(regex, data, re.MULTILINE)
            matches_lst = [i.group(0) for i in matches]
            acordao_number = matches_lst[0].replace(':', '').replace(' ', '')
    except Exception as e:
        pass
    return clean_data(acordao_number)

def extract_acordao(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            acordao = re.search('ACÓRDÃO(.+?)RELATÓRIO', data).group(1)
        elif pattern == 'OLD':
            acordao = re.search('A C Ó R D Ã O(.+?)Documento Assinado', data).group(1)
    except Exception as e:
        print(e)
    return acordao

def extract_date(data):
    acordao = extract_acordao(data)
    try:
        if re.search('(?:Brasilia)(.*)', acordao):
            date = re.search(
                '(?:Brasilia)(.*)', acordao
            ).group(1).replace('(DF)', '').replace(', ', '').replace('\\n', '').replace('.', '')
        else:
            regex = r"(?s)(?<=Brasília)(.*?)(?=\\n)"
            matches = re.finditer(regex, data, re.MULTILINE)
            matches_lst = [i.group(0) for i in matches]
            date = matches_lst[0].replace(' (DF)', '').replace(', ', '')
    except Exception as e:
        pass
    return date

def extract_decision(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            decisao_temp = data.rsplit('DECISÃO')[-1]
            decisao = re.search('(.+?UNÂNIME)', decisao_temp).group(1).replace('\\n', '')
        elif pattern == 'OLD':
            decisao_temp = data.rsplit('D E C I S Ã O')[-1]
            decisao = re.search('(.+?UNÂNIME)', decisao_temp).group(1).replace('\\n', '')
    except Exception as e:
        pass
    return decisao

def extract_ementa(data):
    pattern = check_pdf_pattern(data)
    try:
        if pattern == 'NEW':
            ementa = re.search('EMENTA(.+?)ACÓRDÃO', data).group(1).replace('\\n', '\n')
        elif pattern == 'OLD':
            ementa = re.search('Acórdão N.(.+?)A C Ó R D Ã O', data).group(1).replace('\\n', '\n')
    except Exception as e:
        pass
    return ementa

def extract_ementa_tag(data):
    ementa = extract_ementa(data)
    try:
        regex = r"^(.*?[A-ZÀ-Ú])[A-ZÀ-Ú]\..*"
        matches = re.finditer(regex, ementa, re.MULTILINE)
        matches_lst = [i.group(0) for i in matches]
        ementa_tag = matches_lst
    except Exception as e:
        pass
    return ementa_tag

In [49]:
from tika import parser
from pathlib import *
import pandas as pd

dir_path = Path('D:\\Testes')

orgao_list = []
process_number_list = []
apelante_list = []
apelado_list = []
relator_list = []
acordao_number_list = []
date_list = []
decision_list = []

for i in  list(dir_path.glob('*.pdf')):
    try:
        parsedPDF = parser.from_file(str(i))
        data = str(parsedPDF)
        orgao = extract_orgao(data)
        process_number = extract_process_number(data)
        apelante = extract_apelante(data)
        apelado = extract_apelado(data)
        relator = extract_relator(data)
        acordao_number = extract_acordao_number(data)
        date = extract_date(data)
        decision = extract_decision(data)
        
        orgao_list.append(orgao)
        process_number_list.append(process_number)
        apelante_list.append(apelante)
        apelado_list.append(apelado)
        relator_list.append(relator)
        acordao_number_list.append(acordao_number)
        date_list.append(date)
        decision_list.append(decision)
    except Exception as e:
        pass
    
df = pd.DataFrame({
    'Órgãos': orgao_list,
    'N. Processos': process_number_list,
    'Apelantes': apelante_list,
    'Apelados': apelado_list,
    'Relatores': relator_list,
    'N. Acórdãos': acordao_number_list,
    'Decisões': decision_list,
    'Datas': date_list,
})
#df.to_excel("D:\\Testes\\output.xlsx")  

writer_object = pd.ExcelWriter("D:\\Testes\\Dados.xlsx", engine ='xlsxwriter') 

df.to_excel(writer_object, sheet_name ='Sheet1', index=False) 

workbook = writer_object.book 
worksheet = writer_object.sheets['Sheet1'] 

format1 = workbook.add_format({'num_format': '# 0.00'}) 

worksheet.set_column('A:A', 18, format1)
worksheet.set_column('B:B', 18, format1)
worksheet.set_column('C:C', 50, format1)
worksheet.set_column('D:D', 90, format1)
worksheet.set_column('E:E', 30, format1)
worksheet.set_column('F:F', 35, format1)
worksheet.set_column('G:G', 18, format1)
worksheet.set_column('H:H', 35, format1)

writer_object.save() 

pd.read_excel('D:\\Testes\\Dados.xlsx', index_col=0)  


,N. Processos,Apelantes,Apelados,Relatores,N. Acórdãos,Decisões,Datas
Órgãos,,,,,,,
1ª Turma Cível,APELAÇÃO CÍVEL 0001331-13.2017.8.07.0014,GEAP AUTOGESTÃO EM SAÚDE,GERALDINA ABREU DE SOUZA,TEÓFILO CAETANO,1210449,UNÂNIME,16 de Outubro de 2019
5ª Turma Cível,AGRAVO DE INSTRUMENTO 0715838-09.2019.8.07.0000,HOME - HOSPITAL ORTOPEDICO E MEDICINA ESPECIAL...,IZABEL CARNEIRO RIBEIRO,ANGELO PASSARELI,1211533,CONHECER. NEGAR PROVIMENTO. UNÂNIME,22 de Outubro de 2019
3ª TURMA CÍVEL,APELAÇÃO 20120910158964APC (0015335-46.2012.8....,MARCIA COSTA BATISTA,APEX INCORPORADORA SPE 02 LTDA,MARIO-ZAM BELMIRO,764489,CONHECER. DAR PARCIAL PROVIMENTO AO RECURSO.U...,12 de Fevereiro de 2014
6ª TURMA CÍVEL,APELAÇÃO 20130110448542APC (0012004-46.2013.8....,"LIA ALMEIDA MACHADO PEREIRA, SUL AMÉRICA COMPA...",OS MESMOS,JOSÉ DIVINO,795856,CONHECIDOS. PROVIDA A APELAÇÃO DA AUTORA.DESPR...,4 de Junho de 2014
1ª TURMA CÍVEL,APELAÇÃO 20130111523693APC (0038950-55.2013.8....,GOL LINHAS AEREAS INTELIGENTES S/A,ANDREA PONTES,ALFEU MACHADO,803804,"CONHECER E NEGAR PROVIMENTO, UNÂNIME",17 de Julho de 2014
1ª TURMA CÍVEL,APELAÇÃO 20130110256628APC (0007147-54.2013.8....,"FRANCISCO ERMELINDO ALVES DINIZ, BANCO DAYCOVA...",OS MESMOS,ALFEU MACHADO,803807,NÃO CONHECER DO APELO DO RÉU. CONHECER E DARPA...,17 de Julho de 2014
1ª TURMA CÍVEL,APELAÇÃO 20110112364678APC (0218345-75.2011.8....,NITROGENIUS PRODUTOS QUIMICOS LTDA,HIPER CLEAN COMERCIO E SERVICOS AUTOMOTIVOS LTDA,LEILA ARLANCH,803821,"CONHECER E NEGAR PROVIMENTO, UNÂNIME",10 de Julho de 2014
1ª TURMA CÍVEL,APELAÇÃO 20130110548050APC (0014436-38.2013.8....,"ISABEL DE OLIVEIRA PINTO, ERIKA SOUTO CAMARGO,...",OS MESMOS,LEILA ARLANCH,803839,CONHECER DOS APELOS E NEGAR PROVIMENTO A AMBOS...,10 de Julho de 2014
1ª TURMA CÍVEL,APELAÇÃO 20130110122880APC (0003647-77.2013.8....,SUL AMERICA COMPANHIA DE SEGURO SAUDE,MARIA DAS GRACAS PEREIRA DA COSTA,FLAVIO ROSTIROLA,804033,"CONHECER E NEGAR PROVIMENTO, UNÂNIME",17 de Julho de 2014
